This code is my LSTM portion of the project. You may have to alter the dataset location to run on your machine. I created a shortcut to the shared drive and placed that in mydrive due to dataset size.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import MinMaxScaler

# Load dataset
file_path = "/content/drive/My Drive/cleaned_taxi_data.csv"  # Update if needed
df = pd.read_csv(file_path)

# Convert TIMESTAMP to datetime
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

# Extract time-based features
df['HOUR'] = df['TIMESTAMP'].dt.hour
df['DAY_OF_WEEK'] = df['TIMESTAMP'].dt.dayofweek
df['MONTH'] = df['TIMESTAMP'].dt.month

# Convert POLYLINE (GPS coordinates) from string to list
df['POLYLINE'] = df['POLYLINE'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Remove trips with no GPS data
df = df[df['POLYLINE'].apply(len) > 0]

# Normalize GPS coordinates
scaler = MinMaxScaler()
all_coords = np.vstack(df['POLYLINE'])
scaler.fit(all_coords)

# Function to create LSTM input sequences
def create_sequences(data, seq_length=50):
    X, y = [], []
    for trip in data:
        trip = scaler.transform(trip)  # Normalize coordinates
        if len(trip) > seq_length:
            for i in range(len(trip) - seq_length):
                X.append(trip[i:i+seq_length])  # 50 GPS points as input
                y.append(trip[i+seq_length])    # Next coordinate as output
    return np.array(X), np.array(y)

# Prepare LSTM dataset
X, y = create_sequences(df['POLYLINE'].values)

# Save preprocessed dataset (optional)
np.save("/content/drive/My Drive/X_sequences.npy", X)
np.save("/content/drive/My Drive/y_targets.npy", y)

# Display dataset shape
print("Input Shape (X):", X.shape)  # (samples, 50, 2)
print("Output Shape (y):", y.shape)  # (samples, 2)


Input Shape (X): (9144129, 50, 2)
Output Shape (y): (9144129, 2)


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Load preprocessed data
X = np.load("/content/drive/My Drive/X_sequences.npy")
y = np.load("/content/drive/My Drive/y_targets.npy")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(50, 2)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(2)  # Predict latitude & longitude
])

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train Model
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Save Model
model.save("/content/drive/My Drive/lstm_next_location.h5")

# Evaluate Model
test_loss, test_mae = model.evaluate(X_test, y_test)

# Display results
print("Test Loss (MSE):", test_loss)
print("Test MAE:", test_mae)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambda>

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.

In [ ]:
import matplotlib.pyplot as plt

# Plot loss over epochs
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.title('LSTM Training Performance')
plt.show()


In [ ]:
# Select a random trip from test set
sample_trip = X_test[0].reshape(1, 50, 2)  # Reshape for LSTM input

# Predict next GPS point
predicted_gps = model.predict(sample_trip)

# Convert back to original scale
predicted_gps_original = scaler.inverse_transform(predicted_gps.reshape(1, -1))

print("Predicted Next GPS Coordinate:", predicted_gps_original)
